In [1]:
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/MyDrive/mPDE-Net/')
sys.path.append('/content/drive/MyDrive/mPDE-Net/aTEAM/')
!pip install timeout_decorator
!pip install -U PyYAML

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from sympy.parsing.sympy_parser import parse_expr
from sympy import symbols, lambdify
def givevals(dic, setexp,bs):
  u00="u0"
  v00="v0"
  lnu="lnu"
  I="I"

  newlist={}
  for expr in list(setexp.keys()):


    if expr == 1:
        newlist[expr]=np.ones((200*bs,1))
    elif u00 in str(expr) and v00 in str(expr):

      f = lambdify({u00,v00}, expr)
      finalf=f(dic[u00][:,0],dic[v00][:,0])
      newlist[expr]=finalf.reshape((200*bs,1))
    elif u00 in str(expr) and v00 not in str(expr):

      f = lambdify(u00, expr)
      finalf=f(dic[u00][:,0])
      newlist[expr]=finalf.reshape((200*bs,1))
    elif u00 not in str(expr) and v00 in str(expr):
      f = lambdify(v00, expr)
      finalf=f(dic[v00][:,0])
      newlist[expr]=finalf.reshape((200*bs,1))

    elif "exp"  in str(expr) and "I" not in str(expr):

      f = lambdify(lnu, expr)
      finalf=f(dic[lnu][:,0])

      newlist[expr]=finalf.reshape((200*bs,1))
    elif "exp"  in str(expr) and "I" in str(expr):

      f = lambdify((lnu,I), expr)
      finalf=f(dic[lnu][:,0],dic[I][:,0])
      newlist[expr]=finalf.reshape((200*bs,1))

    else:
      for i in list(dic.keys()):
        if str(i) == str(expr):
          newlist[expr]=dic[i]
  return newlist

def TrainSTRidge(R, Ut, bestu,lam, d_tol, maxit = 35, STR_iters = 30, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.
    It runs over different values of tolerance and trains predictors on a training set, then evaluates them
    using a loss function on a holdout set.
    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.1#*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = bestu
    print(w_best)
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0
    print(err_best)
    print(np.linalg.norm(TestY - TestR.dot(w_best), 2))
    #print(np.linalg.cond(R))

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: print ("Optimal tolerance:", tol_best)

    return w_best, err_best



def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.
    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0

    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]

    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]

        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)

        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0:
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds

        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    if normalize != 0: return np.multiply(Mreg,w)
    else: return w


from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval, STATUS_FAIL
from hyperopt.pyll.base import scope
import numpy as np
space={"sp": hp.loguniform('sp', np.log(1e-7), np.log(1e-4)),
       "msp": hp.loguniform('msp', np.log(1e-7), np.log(1e-4)),
       "stab": hp.loguniform('stab', np.log(1e-7), np.log(1e-4)),
       # "sp0": hp.loguniform('sp0', np.log(0.000000001), np.log(0.000001)),
       # #"msp0": hp.loguniform('msp0', np.log(0.000000001), np.log(0.0001)),
       # "stab0": hp.loguniform('stab0', np.log(0.000000001), np.log(0.000001)),
       }


##insert data
from scipy.optimize.optimize import sqrt
import pandas as pd
import numpy as np
import torch
from numpy import linalg as LA
from scipy.optimize.optimize import sqrt
import pandas as pd
import numpy as np
import torch
from numpy import linalg as LA


<ipython-input-9-02784bb70f40>:175: DeprecationWarning: Please use `sqrt` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import sqrt
<ipython-input-9-02784bb70f40>:180: DeprecationWarning: Please use `sqrt` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import sqrt


In [3]:
datav=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/Ca00510-1.csv",  index_col=0)
datav1=datav.values[0:,:]
datau=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/R00510-1.csv",  index_col=0)
datau1=datau.values[0:,:]


datav2=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int1Ca000110.csv",  index_col=0)
datav2=datav2.values[0:,:]
datau2=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int1R0000110.csv",  index_col=0)
datau2=datau2.values[0:,:]

datav3=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int2Ca000110.csv",  index_col=0)
datav3=datav3.values[0:,:]
datau3=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int2R0000110.csv",  index_col=0)
datau3=datau3.values[0:,:]


datav4=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int3Ca000110.csv",  index_col=0)
datav4=datav4.values[0:,:]
datau4=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int3R0000110.csv",  index_col=0)
datau4=datau4.values[0:,:]


datav5=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int6Ca000110.csv",  index_col=0)
datav5=datav5.values[0:,:]
datau5=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/int6R0000110.csv",  index_col=0)
datau5=datau5.values[0:,:]



datavt=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/testint1Ca00510.csv",  index_col=0)
datavt=datavt.values[0:,:]
dataut=pd.read_csv("/content/drive/MyDrive/mPDE-Net/data/testint1R00510.csv",  index_col=0)
dataut=dataut.values[0:,:]


s=3
datav1=datav1[:,::s]
datau1=datau1[:,::s]

datav2=datav2[:,::s]
datau2=datau2[:,::s]

datav3=datav3[:,::s]
datau3=datau3[:,::s]
datav4=datav4[:,::s]
datau4=datau4[:,::s]

datav5=datav5[:,::s]
datau5=datau5[:,::s]


datavt=datavt[:,::s]
dataut=dataut[:,::s]





def convdata(datau,datav, bs,head,tail,datausize, datavsize):



  if tail==0 and head != 0:
    datau=datau[:head]
    datav=datav[:head]
  elif head==0 and tail !=0:
    datau=datau[tail:]
    datav=datav[tail:]
  elif tail !=0 and head !=0:
    datau=datau[head:tail]
    datav=datav[head:tail]
  else:
    datau=datau
    datav=datav




  lentime=datau.shape[0]
  lenx=datau.shape[1]
  u=[]

  for i in range(0,16):

    u1=datau[i,:]
    v1=datav[i,:]
    u1=torch.from_numpy(u1).to(dtype=torch.float64, device="cuda")
    v1=torch.from_numpy(v1).to(dtype=torch.float64, device="cuda")
    arr=[]
    arr.append(u1)
    arr.append(v1)

    if bs !=1:
      for j in range(bs*2,lentime):
        if j%(bs*2)==1:
          u1=datau[i+j,:]
          v1=datav[i+j,:]
          u1=torch.from_numpy(u1).to(dtype=torch.float64, device="cuda")
          v1=torch.from_numpy(v1).to(dtype=torch.float64, device="cuda")
          arr.append(u1)
          arr.append(v1)
    arr=torch.stack(arr, dim=0)

    arr = arr.view([bs, 2]+[datau.shape[-1]])
    u.append(arr)
  return u



data1=convdata(datau1,datav1,10,0,0,0,0)
data2=convdata(datau2,datav2,1,0,0,0,0)
data3=convdata(datau3,datav3,1,0,0,0,0)
data4=convdata(datau4,datav4,1,0,0,0,0)
data5=convdata(datau5,datav5,1,0,0,0,0)



data=[]
for j in range(len(data3)):
  #a=data3[j]
  a=torch.cat((data1[j], data2[j]))
  c=torch.cat((a, data3[j]))
  d=torch.cat((c, data4[j]))
  e=torch.cat((d, data5[j]))




  data.append(e)

datan=data




datatest=convdata(dataut,datavt,10,0,0,0,0)
datat=[]

for j in range(len(data1)):
  a=datatest[j][:4]



  datat.append(a)


In [4]:
import sys,time
import numpy as np
import torch
import timeout_decorator
from aTEAM.optim import NumpyFunctionInterface
from scipy.optimize.lbfgsb import fmin_l_bfgs_b as lbfgsb
from scipy.optimize.slsqp import fmin_slsqp as slsqp
from scipy.optimize import fmin_bfgs as bfgs
import conf as conf



import setenv1drealalpha as setenv
#import initparameters_1realalpha as initparameters
import initparameters_1realalphano0 as initparameters

kw = {
         '--name':'Murrayll281d',
#         '--dtype':'double',
#         '--device':'cuda:0',
         '--constraint':'2',
#         # computing region
         '--eps':5*np.pi,
         '--dt':5e-2,#
#         '--cell_num':1,
         '--blocks':'0-6,9,12,15',
#         # super parameters of network
         '--kernel_size':19,
#         '--max_order':2,
         '--dx':5*np.pi/200,
         '--hidden_layers':12,
#         '--scheme':'upwind',
#         # data generator
         '--dataname':'Murrayll281d',
#         '--viscosity':0.1,
         '--zoom':1,
         '--max_dt':1e-7,
         '--batch_size': 14,
#         '--data_timescheme':'rk2',
#         '--channel_names':'u,v',
         '--freq':10,
         '--data_start_time':0,
#         # data transform
         '--start_noise':0.01,
         '--end_noise':0.01,#0.01,

      # '--stablize':1.28e-06 ,#0.0
      #    '--sparsity':2.73e-07,
      #    '--momentsparsity':1.64e-06,
      '--stablize':3.94e-06,#0.01
         '--sparsity':1.62e-06,
         '--momentsparsity':1.88e-07,

        '--npseed':-1,
         '--torchseed':-1,
         '--maxiter':1000,
#         '--recordfile':'None',
         '--recordcycle':200,
#         '--savecycle':-1,
#         '--start_from':-1,
          '--mesh_size': [200],
          "--boundary_condition": "Neumann",
          "--lower_bound": -2.5*np.pi,
          "--upper_bound": 2.5*np.pi,
          "--dim":"1d",
          "--multidata":"yes",


         }
options = conf.setoptions(argv=sys.argv[1:],kw=kw,configfile=None)

if torch.cuda.is_available():
      options['--device'] = 'cuda'
else:
      options['--device'] = 'cpu'


globalnames, callback, model, data_model, sampling, addnoise,addnoiset = setenv.setenv(options)

globals().update(globalnames)

torch.cuda.manual_seed_all(torchseed)
torch.manual_seed(torchseed)
np.random.seed(npseed)

  # initialization of parameters
if start_from<0:

      initparameters.initkernels(model, scheme=scheme)
      initparameters.initexpr(model, viscosity=viscosity, pattern='random')

else: # load checkpoint of layer-$start_from
      callback.load(start_from, iternum='final')

ftestloss=[]
# generate data
if options['--multidata']=="yes":
  tu_obs,tu_true,tu = \
        setenv.multidata(datan,globalnames,addnoise,blocks[-1])
elif options['--multidata']=="comb":
  tu_obs,tu_true,tu = \
        setenv.combinedata(model,data_model,datan,globalnames,addnoise,blocks[-1])

else:
  tu_obs,tu_true,tu = \
        setenv.data(datau,datav,globalnames,addnoise,blocks[-1])
tutest_obs,tutest_true,tutest = \
        setenv.multidata(datat,globalnames,addnoise,blocks[-1])
ftestloss=[]
ftrainloss=[]
  #%% train
for block in blocks:
      trainlossf=[]
      testlossf=[]
      if block<=start_from:
          continue

      r = np.random.randn()+torch.randn(1,dtype=torch.float64,device=device).item()
      with callback.open() as output:
          print('device: ', device, file=output)
          print('generate a random number to check random seed: ', r, file=output)
      # print('block: ', block)
      if block == 0:
          callback.stage = 'warmup'
          isfrozen = (False if constraint == 'free' else True)
      else:
          callback.stage = 'block-'+str(block)
          isfrozen = False
          if constraint == 'frozen':
              isfrozen = True
      stepnum = block if block>=1 else 1
      layerweight = [1,]*stepnum

      u_obs,u_true,u = \
        tu_obs[:stepnum+1],tu_true[:stepnum+1],tu[:stepnum+1]
      utest_obs,utest_true,utest = \
        tutest_obs[:stepnum+1],tutest_true[:stepnum+1],tutest[:stepnum+1]

      print("utest_obs", utest_obs[0].shape)
      print("u_obs shape: batchsize x channelNum x xgridsize x ygridsize")
      print(u_obs[0].shape)
      print("u_obs.abs().max()")
      print(u_obs[0].abs().max())

      print("u_lib variance")
      print(initparameters.trainvar(model.UInputs(u_obs[0])))

      print("u_obs variance")
      print(initparameters.trainvar(u_obs[0]))

      # set NumpyFunctionInterface
      def forward():
          stableloss,dataloss,sparseloss,momentloss, testloss = \
                  setenv.loss(model, u_obs, utest_obs, globalnames, block, layerweight)
          #print("dataloss", dataloss)
          layersparsity=0
          if block == 0:
              # for stage='warmup', no regularization term used

              loss = dataloss
          else:
              loss = stablize*stableloss+dataloss+stepnum*sparsity*sparseloss+stepnum*momentsparsity*momentloss#+stepnum*layersparsity*lastlayerloss
          if constraint == 'frozen':
              momentloss = 0
              loss = stablize*stableloss+dataloss+stepnum*sparsity*sparseloss+stepnum*momentsparsity*momentloss#+stepnum*layersparsity*lastlayerloss
          if torch.isnan(loss):
              loss = (torch.ones(1,requires_grad=True)/torch.zeros(1)).to(loss)

          lossr=dataloss.item()
          tloss=testloss

          trainlossf.append(lossr)
          testlossf.append(tloss)

          return loss
      nfi = NumpyFunctionInterface([
          dict(params=model.diff_params(), isfrozen=isfrozen,
              x_proj=model.diff_x_proj, grad_proj=model.diff_grad_proj),
          dict(params=model.expr_params(),
              isfrozen=False)
          ], forward=forward, always_refresh=False,block=block, tol=1)
      callback.nfi = nfi

      def callbackhook(_callback, *args):
          # global model,block,u0_obs,T,stable_loss,data_loss,sparse_loss
          stableloss,dataloss,sparseloss,momentloss, testloss = \
                  setenv.loss(model, u_obs, utest_obs, globalnames, block, layerweight)
          stableloss,dataloss,sparseloss,momentloss = \
                  stableloss.item(),dataloss.item(),sparseloss.item(),momentloss.item()
          with _callback.open() as output:
              print("stableloss: {:.2e}".format(stableloss), "  dataloss: {:.2e}".format(dataloss),
                      "  sparseloss: {:.2e}".format(sparseloss), "momentloss: {:.2e}".format(momentloss),
                      file=output)
          return None
      callbackhookhandle = callback.register_hook(callbackhook)
      if block == 0:
          callback.save(nfi.flat_param, 'start')
      try:

          xopt = bfgs(nfi.f,nfi.flat_param,nfi.fprime,gtol=1e-8,maxiter=maxiter, callback=callback) #,epsilon=1e-6
          # xopt,f,d = lbfgsb(nfi.f, nfi.flat_param, nfi.fprime, m=maxiter, callback=callback, factr=1e7, pgtol=1e-8,maxiter=maxiter,iprint=0)
          np.set_printoptions(precision=2, linewidth=90)
          print("convolution moment and kernels")
          for k in range(max_order+1):

                  print("Moment",(model.__getattr__('fd'+str(k)).moment).data.cpu().numpy())
                  print("keneral",(model.__getattr__('fd'+str(k)).kernel).data.cpu().numpy())
          for p in model.expr_params():
              print("SymNet parameters")
              print(p.data.cpu().numpy())
      except RuntimeError as Argument:
          with callback.open() as output:
              print(Argument, file=output) # if overflow then just print and continue
      finally:
          # save parameters
          nfi.flat_param = xopt
          callback.save(xopt, 'final')
          callback.saveloss(trainlossf, testlossf)
          with callback.open() as output:
              print('finally, finish this stage', file=output)
          callback.record(xopt, callback.ITERNUM)
          callbackhookhandle.remove()
          @timeout_decorator.timeout(1000)
          def printcoeffs():
              with callback.open() as output:
                  print('current expression:', file=output)
                  for poly in model.polys:
                      tsym,csym = poly.coeffs()
                      print(tsym[:20], file=output)
                      print(csym[:20], file=output)
          try:
              printcoeffs()
          except timeout_decorator.TimeoutError:
              with callback.open() as output:
                  print('Time out', file=output)

      ftestloss.append(testlossf)


with callback.open() as output:
      print("u_obs.abs().max()", file=output)
      print(u_obs[0].abs().max(), file=output)
      print("v_obs.abs().max()", file=output)
      print(u_obs[1].abs().max(), file=output)
with torch.no_grad():
      with callback.open() as output:
          print("model(u_obs[0],T=20*dt).abs().max()", file=output)
          print(model(u_obs[0], T=20*dt).abs().max(), file=output)
          print("model(u_obs[0],T=30*dt).abs().max()", file=output)
          print(model(u_obs[0], T=30*dt).abs().max(), file=output)
num_model=[0,1,2,3,4,5,6,7,8,9]
err=[]
for ele in num_model:
    err.append(ftestloss[ele][-1])

print(err)
acc=np.sum(err)/5

<ipython-input-4-94cab508e8dd>:6: DeprecationWarning: Please use `fmin_l_bfgs_b` from the `scipy.optimize` namespace, the `scipy.optimize.lbfgsb` namespace is deprecated.
  from scipy.optimize.lbfgsb import fmin_l_bfgs_b as lbfgsb
<ipython-input-4-94cab508e8dd>:7: DeprecationWarning: Please use `fmin_slsqp` from the `scipy.optimize` namespace, the `scipy.optimize.slsqp` namespace is deprecated.
  from scipy.optimize.slsqp import fmin_slsqp as slsqp


Brusselator
self._dx [0.07853981633974483, 0.07853981633974483]
_FDNd kernel_size [1, 19]
self._dx [0.07853981633974483, 0.07853981633974483]
_FDNd kernel_size [1, 19]
self._dx [0.07853981633974483, 0.07853981633974483]
_FDNd kernel_size [1, 19]
ranum 62301115
torranum 53123836
max_dt 1e-07
last layer
utest_obs torch.Size([4, 2, 200])
u_obs shape: batchsize x channelNum x xgridsize x ygridsize
torch.Size([14, 2, 200])
u_obs.abs().max()
tensor(0.5006, device='cuda:0', dtype=torch.float64)
u_lib variance
tensor(3.7618, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward1>)
u_obs variance
tensor(0.0042, device='cuda:0', dtype=torch.float64)
Optimization terminated successfully.
         Current function value: 0.001661
         Iterations: 368
         Function evaluations: 408
         Gradient evaluations: 408
convolution moment and kernels
Moment [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
keneral [[ 1.80e-17 -2.62e-16  1.85e-15 -8.55e-15  2.98e-14 -8.34e-14  1

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.000330
         Iterations: 209
         Function evaluations: 240
         Gradient evaluations: 228
convolution moment and kernels
Moment [[ 1.00e+00  0.00e+00  7.09e-01 -2.09e-03 -8.60e-03  6.29e-04 -1.49e-02  1.61e-03
   3.30e-02 -3.01e-03 -6.52e-02  2.43e-03  7.92e-02 -5.47e-04 -5.56e-03 -4.10e-04
  -8.12e-02  5.04e-04 -5.85e-02]]
keneral [[-0.01  0.12 -0.63  1.77 -2.75  1.82  0.89 -1.98  0.35  2.08 -0.36 -1.3   0.75  1.57
  -2.48  1.64 -0.6   0.12 -0.01]]
Moment [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
keneral [[-2.29e-06  4.63e-05 -4.50e-04  2.80e-03 -1.26e-02  4.41e-02 -1.27e-01  3.27e-01
  -9.00e-01  5.11e-12  9.00e-01 -3.27e-01  1.27e-01 -4.41e-02  1.26e-02 -2.80e-03
   4.50e-04 -4.63e-05  2.29e-06]]
Moment [[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  2.66e+00 -4.38e-02  2.31e+00 -4.80e-02
   2.54e-01  5.07e-04 -9.95e-01  2.62e-02 -9.33e-01  1.96e-02 -4.70e-01  7.89e-03
  -1.63e-01  2.13e-03 -4.28e-02]]
keneral [[-0.   -0. 

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


...

         Current function value: 0.001882
         Iterations: 195
         Function evaluations: 221
         Gradient evaluations: 209
convolution moment and kernels
Moment [[ 1.    0.    3.66  0.03  1.3   0.05 -0.88 -0.05  0.23  0.02  0.35  0.03 -0.2  -0.02
  -0.31 -0.04 -0.16 -0.02 -0.06]]
keneral [[-6.64e-04 -7.83e-03  7.46e-02 -1.59e-01 -5.35e-02  3.23e-01  3.76e-01 -1.84e-02
  -7.83e-01  1.41e+00 -6.06e-01 -5.48e-02  3.23e-01  2.58e-01  3.87e-02 -1.66e-01
   4.72e-02  4.90e-03 -2.44e-03]]
Moment [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
keneral [[-2.29e-06  4.63e-05 -4.50e-04  2.80e-03 -1.26e-02  4.41e-02 -1.27e-01  3.27e-01
  -9.00e-01  5.11e-12  9.00e-01 -3.27e-01  1.27e-01 -4.41e-02  1.26e-02 -2.80e-03
   4.50e-04 -4.63e-05  2.29e-06]]
Moment [[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  4.63e+00 -2.21e-02  8.81e+00 -4.62e-02
   9.38e+00 -3.82e-02  6.46e+00 -1.49e-02  3.13e+00 -1.23e-03  1.13e+00  1.67e-03
   3.17e-01  1.02e-03  7.12e-02]]
keneral [[ 0.    0.0

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.002506
         Iterations: 164
         Function evaluations: 192
         Gradient evaluations: 180
convolution moment and kernels
Moment [[ 1.    0.    3.67  0.04  1.39  0.03 -0.97 -0.04  0.32  0.02  0.3   0.01 -0.24 -0.02
  -0.26 -0.02 -0.1  -0.01 -0.02]]
keneral [[ 0.   -0.03  0.17 -0.32  0.01  0.43  0.39 -0.25 -0.83  1.77 -0.64 -0.27  0.23  0.44
   0.15 -0.42  0.2  -0.04  0.  ]]
Moment [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
keneral [[-2.29e-06  4.63e-05 -4.50e-04  2.80e-03 -1.26e-02  4.41e-02 -1.27e-01  3.27e-01
  -9.00e-01  5.11e-12  9.00e-01 -3.27e-01  1.27e-01 -4.41e-02  1.26e-02 -2.80e-03
   4.50e-04 -4.63e-05  2.29e-06]]
Moment [[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  4.71e+00 -3.19e-02  9.07e+00 -5.87e-02
   9.74e+00 -3.08e-02  6.74e+00  9.30e-03  3.28e+00  2.08e-02  1.18e+00  1.34e-02
   3.33e-01  5.36e-03  7.49e-02]]
keneral [[ 0.    0.01  0.01  0.    0.   -0.   -0.   -0.   -0.   -0.02 -0.   -0.   -0.   -0.
   0. 

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 0.003109
         Iterations: 115
         Function evaluations: 135
         Gradient evaluations: 123
convolution moment and kernels
Moment [[ 1.00e+00  0.00e+00  3.69e+00  4.17e-02  1.38e+00  2.26e-02 -9.36e-01 -3.31e-02
   2.79e-01  2.53e-02  3.21e-01  1.24e-03 -2.22e-01 -1.68e-02 -2.78e-01 -6.74e-03
  -1.28e-01  4.56e-04 -3.63e-02]]
keneral [[-1.75e-04 -1.23e-02  8.92e-02 -1.75e-01 -6.16e-02  3.01e-01  5.76e-01 -2.58e-01
  -9.56e-01  1.90e+00 -7.97e-01 -1.69e-01  2.92e-01  3.59e-01  1.34e-01 -3.60e-01
   1.62e-01 -2.64e-02  8.95e-04]]
Moment [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
keneral [[-2.29e-06  4.63e-05 -4.50e-04  2.80e-03 -1.26e-02  4.41e-02 -1.27e-01  3.27e-01
  -9.00e-01  5.11e-12  9.00e-01 -3.27e-01  1.27e-01 -4.41e-02  1.26e-02 -2.80e-03
   4.50e-04 -4.63e-05  2.29e-06]]
Moment [[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  4.69e+00 -1.31e-02  9.06e+00 -2.78e-02
   9.81e+00 -2.20e-02  6.88e+00 -6.84e-03  3.40e+00  1.19e

In [5]:
import shutil
import os

folder_name = 'Murrayll281d'

# Create a ZIP file containing the folder
shutil.make_archive(folder_name, 'zip', "/content/checkpoint/Murrayll281d")


'/content/Murrayll281d.zip'

In [10]:
#Retrieve Coefficients from Trained mFrac-PDE-Net Model


options = {}
options['--name'] = 'Murrayll281d'
options['--dim']="1d"
configfile="checkpoint"+"/"+options['--name']+'/options.yaml'
options = conf.setoptions(argv=None,kw=None,configfile=configfile,isload=True)
options['--device'] = 'cuda'
block = 15

globalnames, callback, model, data_model, sampling, addnoise,addnoiset = setenv.setenv(options)

callback.load(int(block))

tsym,csym = model.poly0.coeffs(calprec=6)
terms0 = tsym[:40]
coeffs0 = np.zeros((len(terms0)))
coeffs0[:min(len(tsym),40)] = csym[:min(len(tsym),40)]

tsym,csym = model.poly1.coeffs(calprec=6)
terms1 = tsym[:40]
coeffs1 = np.zeros((len(terms1)))
coeffs1[:min(len(tsym),40)] = csym[:min(len(tsym),40)]
torch.save(dict(terms0=terms0,terms1=terms1,coeffs0=coeffs0,coeffs1=coeffs1),'-block-000-'+str(block))



l1=terms0
l2=terms1
coe1=coeffs0
coe2=coeffs1


T = 0.05
u = tu_obs
init = u[0]
x0 = init

utest=tutest_obs#
initest=utest[0]
xtest=initest

bst=15
nini=14
bs=bst*nini
x1=x0
xtemp=np.zeros((bs,8,200))
utemp=np.zeros((bs,2,200))

xtempt=np.zeros((bs,8,200))
utempt=np.zeros((bs,2,200))
uutempt=np.zeros((bs,1,200))
uvtempt=np.zeros((bs,1,200))



for t in range(bst):
      print('t={:.1e}'.format(t*T))

      startt = time.time()
      with torch.no_grad():
          x11 = model.predict(x1, T)

          xtmp4, ut = model.predict_val(x1,T)
          xtmp4=xtmp4[:nini,...,...]
          ut=ut[:nini,...,...]
          xtemp[nini*t:nini*(t+1),:,:]=xtmp4.data.cpu().numpy()
          utemp[nini*t:nini*(t+1),:,:]=ut.data.cpu().numpy()
          x1=u[t+1]#x11

          x1test = model.predict(xtest, T)

          xtmp4t, utt = model.predict_val(xtest,T)
          xtmp4t=xtmp4t[:1,...,...]
          utt=utt[:1,...,...]
          xtempt[t,:,:]=xtmp4t.data.cpu().numpy()
          utempt[t,:,:]=utt.data.cpu().numpy()
          xtest=utest[t+1]#x1test



        #print("x1", x1)
      stopt = time.time()





allchannels = []
channel_names=model.channel_names
max_order=2

for c in channel_names:
            for k in range(max_order+1):
                    allchannels.append(c+str(k))
allchannels.append("I")
allchannels.append("lnu")


xnew=xtemp

xnewt=xtempt

#get matrix of X
channel_dic={}
for order, ele in enumerate(xnew[0]):
    channel_dic[allchannels[order]]=xnew[:,order,:].reshape(200*bs,1)

channel_dict={}
for order, ele in enumerate(xnew[0]):
    channel_dict[allchannels[order]]=xnewt[:,order,:].reshape(200*bs,1)



rhsu={i:0 for i in l1}
rhsv={i:0 for i in l2}

rhu1=givevals(channel_dic, rhsu, bs)
rhv1=givevals(channel_dic, rhsv,bs)

rhsut={i:0 for i in l1}
rhsvt={i:0 for i in l2}

rhut=givevals(channel_dict, rhsut, bs)
rhvt=givevals(channel_dict, rhsvt,bs)



dataMatrixu = np.array([rhu1[i] for i in l1])
dataMatrixv = np.array([rhv1[i] for i in l2])



linear_matu=dataMatrixu.T.reshape((200*bs,len(l1)))
linear_matv=dataMatrixv.T.reshape((200*bs,len(l2)))


coefu=coeffs0
coefv=coeffs1

coefu=np.reshape(coefu, (-1,1))
coefv=np.reshape(coefv, (-1,1))
linear_matu1=np.round(linear_matu,4)
linear_matv1=np.round(linear_matv,4)

uut=linear_matu1@coefu
uvt=linear_matv1@coefv

#l2
#u
l2normu=np.sum(np.abs(linear_matu.T)**2,axis=-1)**(1./2)
l2normu1=l2normu.reshape((len(l2normu),1))
scalu=l2normu1*coefu
indtermusp=[]
for order,ele in enumerate(scalu):
    print(ele)
    if abs(ele)>max(abs(scalu))/100:

      indtermusp.append(order)

luspy=[]
for i in indtermusp:
    luspy.append(l1[i])


#v
l2normv=np.sum(np.abs(linear_matv.T)**2,axis=-1)**(1./2)
l2normv1=l2normv.reshape((len(l2normv),1))
scalv=l2normv1*coefv


indtermvsp=[]
for order,ele in enumerate(scalv):

    if abs(ele)>max(abs(scalv))/100:

      indtermvsp.append(order)

lvspy=[]
for i in indtermvsp:
    lvspy.append(l2[i])

#Sparsity
#u
dataMatrixuy = np.array([rhu1[i] for i in luspy])

linear_matuy=dataMatrixuy.T.reshape((200*bs,len(luspy)))

dataMatrixuyt = np.array([rhut[i] for i in luspy])

linear_matuyt=dataMatrixuyt.T.reshape((200*bs,len(luspy)))


coefuspy=[]

for i in indtermusp:
    coefuspy.append(coefu[i])

linear_matuf1=np.round(linear_matuy,4)
linear_matuf1t=np.round(linear_matuyt,4)

coefuspy=np.reshape(coefuspy, (-1,1))


 #v
dataMatrixvy = np.array([rhv1[i] for i in lvspy ])

linear_matvy=dataMatrixvy.T.reshape((200*bs,len(lvspy)))

dataMatrixvyt = np.array([rhvt[i] for i in lvspy ])

linear_matvyt=dataMatrixvyt.T.reshape((200*bs,len(lvspy)))


coefvspy=[]

for i in indtermvsp:
    coefvspy.append(coefv[i])

linear_matvf1=np.round(linear_matvy,4)
linear_matvf1t=np.round(linear_matvyt,4)

coefvspy=np.reshape(coefvspy, (-1,1))



ucoef=TrainSTRidge(linear_matuf1,uut,coefuspy,0, 10)
vcoef=TrainSTRidge(linear_matvf1,uvt,coefvspy,50, 10)



utestf=linear_matuf1t@ucoef[0]
vtestf=linear_matvf1t@vcoef[0]

utestf=utestf[:(bs-1)*200]
vtestf=vtestf[:(bs-1)*200]

#record terms and coe with sparsity

finalindu=list(np.where(ucoef[0]!=0)[0])
fl1=[]
fcoeu=[]

for i in finalindu:
    fl1.append(luspy[i])

    fcoeu.append(ucoef[0][i].real)



finalindv=list(np.where(vcoef[0]!=0)[0])
fl2=[]
fcoev=[]

for i in finalindv:

    fl2.append(lvspy[i])

    fcoev.append(vcoef[0][i].real)




self._dx [0.07853981633974483, 0.07853981633974483]
_FDNd kernel_size [1, 19]
self._dx [0.07853981633974483, 0.07853981633974483]
_FDNd kernel_size [1, 19]
self._dx [0.07853981633974483, 0.07853981633974483]
_FDNd kernel_size [1, 19]
ranum 53280038
torranum 13732816
max_dt 1e-07
t=0.0e+00
t=5.0e-02
t=1.0e-01
t=1.5e-01
t=2.0e-01
t=2.5e-01
t=3.0e-01
t=3.5e-01
t=4.0e-01
t=4.5e-01
t=5.0e-01
t=5.5e-01
t=6.0e-01
t=6.5e-01
t=7.0e-01
[52.17]
[-93.79]
[-18.28]
[2.2]
[77.79]
[-0.31]
[0.]


<ipython-input-9-02784bb70f40>:17: SymPyDeprecationWarning: 

Passing the function arguments to lambdify() as a set is deprecated. This
leads to unpredictable results since sets are unordered. Instead, use a list
or tuple for the function arguments.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-lambdify-arguments-set
for details.

This has been deprecated since SymPy version 1.6.3. It
will be removed in a future version of SymPy.

  f = lambdify({u00,v00}, expr)


[[ 3.58]
 [-3.4 ]
 [-0.26]
 [ 0.11]
 [ 0.07]]
0.6399917913890403
0.13999179138904022


<ipython-input-9-02784bb70f40>:127: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  else: w = np.linalg.lstsq(X,y)[0]
<ipython-input-9-02784bb70f40>:153: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
<ipython-input-9-02784bb70f40>:156: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to k

[[ 9.38]
 [ 0.96]
 [-0.4 ]]
0.3000000000000008
7.667108016084408e-16


<ipython-input-9-02784bb70f40>:126: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]


In [13]:
#mfrac-pde-net
print(l1)
print(l2)
print(coe1)
print(coe2)

[exp(0.750087930051892*lnu), u0, I*exp(0.750087930051892*lnu), u2, u0/(1.0*v0**2 - 0.00103095425807703*v0 + 0.020545923978315), v0, u0*v0/(1.0*v0**2 - 0.00103095425807703*v0 + 0.020545923978315)]
[v2, u0, v0]
[ 3.58e+00 -3.40e+00 -2.60e-01  1.10e-01  7.07e-02 -7.92e-03  3.30e-05]
[ 9.38  0.96 -0.4 ]


In [15]:
#l2 norm + sp
print(luspy)
print(lvspy)
print(coefuspy)
print(coefvspy)

[exp(0.750087930051892*lnu), u0, I*exp(0.750087930051892*lnu), u2, u0/(1.0*v0**2 - 0.00103095425807703*v0 + 0.020545923978315)]
[v2, u0, v0]
[[ 3.58]
 [-3.4 ]
 [-0.26]
 [ 0.11]
 [ 0.07]]
[[ 9.38]
 [ 0.96]
 [-0.4 ]]
